In [55]:
from bs4 import BeautifulSoup
import requests
import re
from datetime import datetime, timedelta
from dateutil import tz

In [85]:
req = requests.get(start_url)

In [86]:
soup = BeautifulSoup(req.content, 'html.parser')

In [87]:
news_items = soup.find("div", id="page").find_all(class_='list-item')

In [88]:
print(len(news_items))

20


In [95]:
base_url = 'https://ria.ru'
start_urls = [
    f'{base_url}/category_rasprostranenie-novogo-koronavirusa+location_Finland/',
    f'{base_url}/category_rasprostranenie-novogo-koronavirusa+location_Estonia/',
    f'{base_url}/category_rasprostranenie-novogo-koronavirusa+location_Sweden/',
]

current_tz = tz.gettz('Europe/Moscow')
dt_format = '%H:%M %d.%m'

countries_news = []
for start_url in start_urls:
    req = requests.get(start_url)
    soup = BeautifulSoup(req.content, 'html.parser')
    news_items = soup.find("div", id="page").find_all(class_='list-item')

    for news_item in news_items:
        schema_h = news_item.find(class_='schema_org')
        link = schema_h.find(attrs={"itemprop": "url"}).get('content')
        title = schema_h.find(attrs={"itemprop": "name"}).get('content')
        human_dt = news_item.find(class_='list-item__date').get_text()
        dt = parse_datetime(human_dt)
        
        countries_news.append({
            'title': title,
            'link': f'{base_url}{link}',
            'published_origin': human_dt,
            'published': dt.strftime(dt_format),
            'published_dt': dt
        })

In [96]:
print(countries_news)

[{'title': 'Восемь человек с коронавирусом выздоровели в Хельсинки', 'link': 'https://ria.ru/20200314/1568615319.html', 'published_origin': 'Вчера, 21:41', 'published': '21:41 14.03', 'published_dt': datetime.datetime(2020, 3, 14, 21, 41, tzinfo=tzfile('/usr/share/zoneinfo/Europe/Moscow'))}, {'title': 'В Финляндии число случаев коронавируса увеличилось до 210 человек', 'link': 'https://ria.ru/20200314/1568600195.html', 'published_origin': 'Вчера, 13:44', 'published': '13:44 14.03', 'published_dt': datetime.datetime(2020, 3, 14, 13, 44, tzinfo=tzfile('/usr/share/zoneinfo/Europe/Moscow'))}, {'title': 'Посольство уточнило число россиян на лайнере Costa Magica', 'link': 'https://ria.ru/20200313/1568577379.html', 'published_origin': '13 марта, 18:47', 'published': '18:47 13.03', 'published_dt': datetime.datetime(2020, 3, 13, 18, 47, tzinfo=tzfile('/usr/share/zoneinfo/Europe/Moscow'))}, {'title': 'В Гонконге рекомендовали избегать поездок в ряд стран Шенгенской зоны', 'link': 'https://ria.ru

In [92]:
st = 'Вчера, 21:41'
st = '5 апреля, 03:41'

def parse_datetime(st):
    monthes_map = ['января', 'февраля', 'марта', 'апреля', 'мая', 'июня', 
                   'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря']

    time_part = re.findall('\d{2}:\d{2}', st)[0].split(':')
    hour = int(time_part[0])
    minutes = int(time_part[1])
    if 'Вчера' in st:
        dt = (datetime.now(tz=current_tz) - timedelta(days=1)).replace(
            hour=hour, minute=minutes, second=0, microsecond=0
        )
    elif ',' in st:
        datepart = re.findall('(.+),', st)[0].split()
        day = int(datepart[0])
        month = int(monthes_map.index(datepart[1])) + 1
        year = 2020
        dt = datetime(year=year, month=month, day=day, hour=hour, minute=minutes, tzinfo=current_tz)
    else:
        dt = datetime.now(tz=current_tz).replace(
            hour=hour, minute=minutes, second=0, microsecond=0
        )
    return dt
    
    
print(parse_datetime('Вчера, 01:41'))
print(parse_datetime('01:41'))
print(parse_datetime('16 апреля, 23:41'))

2020-03-14 01:41:00+03:00
2020-03-15 01:41:00+03:00
2020-04-16 23:41:00+03:00
